In [1]:
from env.pok_env_gym_RLLib import PokemonEnv
from ray.tune.registry import register_env


/Users/jskaf/miniconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
def env_creator(arg):
    print(arg)
    return PokemonEnv()

register_env("PokemonEnv", env_creator)

In [3]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig, DQN
from agent.DQNGAN import GAN_DQN
from gan.GANExploration import GANExploration
from ray.tune.logger import pretty_print
import ray

input_size = (120,120)
ray.init()

replay_config = {
        "type": "MultiAgentPrioritizedReplayBuffer",
        "capacity": 60000,
        "prioritized_replay_alpha": 0.5,
        "prioritized_replay_beta": 0.5,
        "prioritized_replay_eps": 3e-6,
    }

config = (DQNConfig()
          .training(replay_buffer_config=replay_config,
                         train_batch_size=32,
                         n_step=1,
                         num_atoms=1,
                         double_q=False,
                         dueling=False,
                         num_steps_sampled_before_learning_starts=100)
            .resources(num_gpus=0)
            .rollouts(num_rollout_workers=1)
            .environment("PokemonEnv")
            .exploration(exploration_config={
                "type": GANExploration,
                "file_path_for_gan_weights":"",
                "input_size": input_size,
            }))
algo = GAN_DQN(config=config)
for i in range(1):
   print(pretty_print(algo.train())) 
ray.shutdown()

2024-03-23 14:41:20,277	INFO worker.py:1724 -- Started a local Ray instance.
2024-03-23 14:41:20,652	WARNING deprecation.py:50 -- DeprecationWarning: `rllib/algorithms/simple_q/` has been deprecated. Use `rllib_contrib/simple_q/` instead. This will raise an error in the future!
/Users/jskaf/miniconda3/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:483: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/Users/jskaf/miniconda3/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json

(RolloutWorker pid=76207) {, worker=1/1, vector_idx=0, remote=False}


(RolloutWorker pid=76207) 2024-03-23 14:41:22,706	WARNING env.py:162 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.


(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0


2024-03-23 14:41:24,590	WARNING util.py:62 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels

2024-03-23 14:41:28,087	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels: 0, exploration: 0
(RolloutWorker pid=76207) Pokédex: 0, Badges: 0, Death: 0, Levels

In [4]:
ray.shutdown()